In [1]:
#!/usr/bin/env python
# coding: utf8
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from tqdm.auto import tqdm

In [2]:
from spacy.gold import GoldCorpus

In [3]:
import os
os.environ['embed_size'] = '8000'

In [4]:
CFG = {'device': 1, 'cpu_count': 4}
TESTS = False
spacy.require_gpu()

True

In [5]:
import gzip, json
def load_entries(fn): # '../data/datasets/nerus.jsonl.gz'
    entries = []
    with gzip.open(fn, 'r') as f:
        for line in tqdm(f):
            entry = json.loads(line)
            entries.append(entry)
    return entries
    #del entries

In [7]:
g=GoldCorpus('../../data/syntagrus/ru_syntagrus-ud-train.json', 
             '../../data/syntagrus/ru_syntagrus-ud-test.json')

In [8]:
g.limit = None

In [9]:
nlp_ = spacy.blank('ru')
d, gp = next(g.train_docs(nlp_))
gp.tags

['ADJ__Animacy=Inan|Case=Acc|Degree=Pos|Number=Plur',
 'NOUN__Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur',
 'NOUN__Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing',
 'ADJ__Case=Gen|Degree=Pos|Gender=Fem|Number=Sing',
 'NOUN__Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing',
 'VERB__Aspect=Imp|Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act',
 'ADP___',
 'NOUN__Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur',
 'PUNCT___']

In [10]:
for i in g.train_docs(nlp_):
    print(i)
    break

(Так все равно у нас с 2000 года не велось реальной подготовки к вступлению в ВТО , и мы к этому не готовы так же , как и восемь лет назад . , <spacy.gold.GoldParse object at 0x7f5714639048>)


In [11]:
from importlib import reload
import utils.corpus
reload(utils.corpus)

<module 'utils.corpus' from '/Projects/nlp/spacy/spacy-ru/utils/corpus.py'>

In [12]:
from utils.corpus import Corpus, tag_morphology, iter_corpora

In [13]:
SynTagRus = Corpus.from_gold('ru', g)

In [14]:
assert len(list(iter_corpora([SynTagRus]))) == 2

In [15]:
print(len(SynTagRus.ds_train), len(SynTagRus.ds_test))
print(len(SynTagRus.ds_train.pos), len(SynTagRus.ds_test.pos))
print(len(SynTagRus.ds_train.dep), len(SynTagRus.ds_test.dep))
print(len(SynTagRus.ds_train.ner), len(SynTagRus.ds_test.ner))

48814 6491
733 608
294 39
0 0


In [15]:
USE_RU2 = False
if USE_RU2:
    import ru2e
    nlp = ru2e.load_ru2('../../ru2e/')
else:
    nlp = spacy.blank('ru')
nlp.pipeline

[]

In [16]:
nlp.vocab.morphology.tag_map.clear()
nlp.vocab.morphology.tag_map

{}

In [17]:
def add_submodel(nlp, name, rebuild=True):
    if name in nlp.pipe_names and rebuild:
        nlp.disable_pipes(name)
    if name not in nlp.pipe_names:
        print("Creating new submodel for {}...".format(name))
        submodel = nlp.create_pipe(name)
        nlp.add_pipe(submodel)
    submodel = nlp.get_pipe(name)
    return submodel

In [32]:
def setup_ner(nlp, *corpora, rebuild=True):
    ner = add_submodel(nlp, 'ner', rebuild=rebuild)
    for c in iter_corpora(corpora):
        for l in c.ents:
            ner.add_label(l)

def setup_tagger(nlp, *corpora, rebuild=True):
    pos = add_submodel(nlp, 'tagger', rebuild=rebuild)
    for c in iter_corpora(corpora):
        for l in c.pos:
            pos.add_label(l, tag_morphology(l))
    print(len(pos.labels))

def setup_parser(nlp, *corpora, rebuild=True):
    dep = add_submodel(nlp, 'parser', rebuild=rebuild)
    for c in iter_corpora(corpora):
        for l in c.dep:
            dep.add_label(l)
    print(len(dep.labels))

setup_tagger(nlp, SynTagRus, rebuild=True)
setup_parser(nlp, SynTagRus, rebuild=True)
#setup_ner(nlp, SynTagRus, rebuild=True)
print(f"Morphology tag map: {len(nlp.vocab.morphology.tag_map)}")
nlp.pipeline

Creating new submodel for tagger...
745
Creating new submodel for parser...
294
Morphology tag map: 745


[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f62f18a35c0>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f633453b2e8>)]

In [33]:
TESTS = 0

In [34]:
from utils.pluck import pluck, pluck_list, pluck_dict
from utils.tqdm import tqdm_batches

In [35]:
from IPython.core.debugger import set_trace

In [36]:
import spacy
import random
from spacy.gold import GoldParse
from spacy.scorer import Scorer
import pandas
pandas.set_option('display.precision', 3) 

def init_model(model, CFG):
    for n, m in model.pipeline:
        if m.model is True:
            print(f"Initializing model because of {n}!")
            model.begin_training(**CFG)
            break

def _evaluate(model, batches):
    scorer = Scorer()
    for batch in batches:
        # print("Example batch:", batch)
        orig_docs, golds = zip(*batch)
        docs = model.pipe([b.text for b in orig_docs])
        for doc, parse in zip(docs, golds):
            scorer.score(doc, parse)
    return scorer.scores

def evaluate(model, dataset, limit=None, batch_size=32):
    generator = dataset.iter(model, limit=limit)
    batches = tqdm_batches(minibatch(generator, batch_size), total=limit or len(dataset), leave=False)
    return _evaluate(model, batches)

def evaluate_data_source(model, ds, count=None, batch_size=32):
    # enable_entities(model, ds.ds_test.ner)
    res = evaluate(model, ds.ds_test, limit=count)
    return res

def get_ent_scores(res):
    return {k:v for k,v in res['ents_per_type'].items() if k in ds.ents}

def display_scores(list_of_scores):
    if isinstance(list_of_scores, dict):
        list_of_scores = [list_of_scores]
    display(pandas.DataFrame.from_records(list_of_scores))

def display_scores_t(list_of_scores):
    if isinstance(list_of_scores, dict):
        list_of_scores = [list_of_scores]
    display(pandas.DataFrame.from_records(list_of_scores).T)

if TESTS or 0:
    init_model(nlp, CFG)
    scores = evaluate_data_source(nlp, SynTagRus, count=None, batch_size=32)
    display_scores(scores)

In [66]:
import numpy

def get_other_pipes(nlp, *x):
    return [pipe for pipe in nlp.pipe_names if pipe not in x]

def get_optimizer(model, CFG):
    init_model(model, CFG)
    if not hasattr(model, '_optimizer'):
        model._optimizer = model.resume_training(**CFG)
    return model._optimizer

def _train_epoch(model, labels, batches):
    init_model(model, CFG)
    with model.disable_pipes(*get_other_pipes(model, 'tagger', 'parser')):
        # print("Training only:", model.pipe_names)
        optimizer = get_optimizer(model, CFG)
        losses = {}
        n_docs = 0
        for batch in batches:
            texts, anns = zip(*batch)
            # enable_entities(model, labels)
            model.update(texts, anns, drop=0.2, losses=losses, sgd=optimizer)
            n_docs += len(batch)
        meta = {'loss_'+k: numpy.log(1e-10 + (v / n_docs)) for k,v in losses.items()}
        meta['docs'] = n_docs
    # enable_all_entities(model)
    return meta

def train_epoch(model, ds, batch_size, count=None):
    batches = minibatch(ds.iter(model, limit=count), size=size_)
    return _train_epoch(model, ds.ner, tqdm_batches(batches, total=count or len(ds), leave=False, drp=0.2))

if TESTS or 0:
    size_ = compounding(1., 32., 1.001)
    meta = train_epoch(nlp, SynTagRus.ds_train, batch_size=size_, count=1000)
    display_scores(meta)

if TESTS or 0:
    display_scores(evaluate_data_source(nlp, SynTagRus))    

In [67]:
if TESTS or 0:
    scorer = spacy.scorer.Scorer()
    for doc, parse in tqdm(SynTagRus.ds_test.iter(nlp, limit=1000), total=1000):
        doc = nlp(doc.text)
        #explacy.print_parse_info(nlp, doc.text)
        #parse.tags = [t.split('__', 1)[0] for t in parse.tags]
        #print([(dt.tag_, pt) for dt,pt in zip(doc, parse.tags)])
        scorer.score(doc, parse, verbose=False)
    display_scores(scorer.scores)
    #display_scores(evaluate_data_source(nlp, SynTagRus, count=1000))

In [68]:
import spacy
from notebooks.examples import explacy
if TESTS or 0:
    doc = nlp(u'Жизнь после двух тысяч первого года на Марсе стала сложней.')
    explacy.print_parse_info(nlp, doc.text)
    explacy.print_parse_info(nlp, d.text)

In [69]:
CORPORA = [SynTagRus]

In [71]:
g.limit = None
size_ = compounding(16., 32., 1.0005)
for e in tqdm(range(22, 30)):
    res = {'epoch': e+1}
    for c in CORPORA:
        meta = train_epoch(nlp, c.ds_train, batch_size=size_, count=len(c.ds_train)//2)
        for k,v in sorted(meta.items()):
            res[k] = v
    for c in CORPORA:
        res.update(evaluate_data_source(nlp, c, count=None))
    avg = res.copy()
    with nlp.use_params(get_optimizer(nlp, CFG).averages):
        for c in CORPORA:
            avg.update(evaluate_data_source(nlp, c, count=None))
    display_scores([res, avg])

KeyboardInterrupt: 

In [ ]:
from pathlib import Path
# save model to output directory
def save_model(nlp, output_dir):
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)
save_model(nlp, '../../ru2_pos_dep_wider_bsz16')

In [ ]:
nlp.tokenizer('приветы всем'.split())

In [ ]:
spacy.displacy.render(nlp('20 декабря 2019 года на улице Советской, город Новосибирск, мы с Сашей пошли гулять'), 
                      style='ent')